# Loading the data from HuggingFace

In [ ]:
from datasets import load_dataset
dataset = load_dataset("nazimali/kurdish-english-opus-100")
train_20k = dataset["train"].shuffle(seed=42).select(range(20000))



# Loading the base nllb model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "facebook/nllb-200-distilled-600M"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


# Data Tokenizing function

In [15]:
max_input_length = 128
max_target_length = 128

def tokenize(batch):
    model_inputs = tokenizer(
        batch["english"],
        padding="max_length",
        truncation=True,
        max_length=max_input_length
    )

    labels = tokenizer(
        batch["kurdish"],
        padding="max_length",
        truncation=True,
        max_length=max_target_length
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# Applying the Tokenizing function

In [16]:
tokenized_dataset = train_20k.map(
    tokenize,
    batched=True,
    remove_columns=train_20k.column_names
)



Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

# Setting up the lora configurations

In [17]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r = 8,
    lora_alpha = 16,
    lora_dropout = 0.05,
    bias = "none",
    task_type = "SEQ_2_SEQ_LM",
    target_modules=[
        "q_proj", "k_proj", "v_proj", "out_proj"
    ]
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 2,359,296 || all params: 617,433,088 || trainable%: 0.3821


# Importing libraries for training

In [18]:
from transformers import (
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    GenerationConfig,
    DataCollatorForSeq2Seq
)


# Setting up the training arguments

In [19]:
gen_config = GenerationConfig.from_pretrained(model_name)

training_args = Seq2SeqTrainingArguments(
    output_dir="nllb_ckb_lora",
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs=2,
    weight_decay=0.01,
    fp16=True,
    logging_steps=50,
    save_total_limit=1,
    save_steps=500,
    generation_config=gen_config
)

# Setting up the data collator and trainer

In [21]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    processing_class=tokenizer,
)

# Training the model

In [22]:
trainer.train()


Step,Training Loss
50,0.654400
100,0.531200
150,0.554400
200,0.529100
250,0.570600
300,0.545000
350,0.573700
400,0.557200
450,0.585600
500,0.572200


TrainOutput(global_step=5000, training_loss=0.6155603992462159, metrics={'train_runtime': 2068.9435, 'train_samples_per_second': 19.334, 'train_steps_per_second': 2.417, 'total_flos': 1.090800058368e+16, 'train_loss': 0.6155603992462159, 'epoch': 2.0})

# Saving the fine tuned model

In [23]:
model.save_pretrained("nllb_kurdish_lora_adapter")
